In [8]:
!git clone https://github.com/NitheshYetikuri/FurnitureStore-backend.git
!cd FurnitureStore-backend/furniturestore

Cloning into 'FurnitureStore-backend'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 190 (delta 36), reused 161 (delta 33), pack-reused 0 (from 0)
Receiving objects: 100% (190/190), 127.88 KiB | 1.28 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [62]:
!pip install astrapy javalang langchain-astradb langchain_google_genai

In [80]:
import os
import javalang
import json

def parse_java_files(directory):
    metadata = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".java"):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r') as f:
                        java_code = f.read()
                        tree = javalang.parse.parse(java_code)
                        class_info = {
                            "class_name": None,
                            "file_path": file_path,
                            "methods": [],
                            "attributes": [],
                            "dependencies": []
                        }
                        for path, node in tree:
                            if isinstance(node, javalang.tree.ClassDeclaration) or isinstance(node, javalang.tree.InterfaceDeclaration):
                                class_info["class_name"] = node.name
                                if hasattr(node, "fields") and node.fields: #check if fields exist.
                                    for field in node.fields:
                                        for declarator in field.declarators:
                                            class_info["attributes"].append(declarator.name)
                            elif isinstance(node, javalang.tree.MethodDeclaration):
                                method_info = {
                                    "name": node.name,
                                    "parameters": [param.name for param in node.parameters],
                                }
                                class_info["methods"].append(method_info)
                            elif isinstance(node, javalang.tree.Import):
                                class_name = node.path.split('.')[-1]
                                class_info["dependencies"].append(class_name)
                        if class_info["class_name"] is None:
                            for path, node in tree.filter(javalang.tree.ClassDeclaration, javalang.tree.InterfaceDeclaration):
                                class_info["class_name"] = node.name
                                if hasattr(node, "documentation") and node.documentation: #check if doc exists.
                                    class_info["docstring"] = node.documentation
                        metadata.append(class_info)
                except Exception as e:
                    print(f"Error processing file {file_path}: {e}")
    return metadata

directory = '/content/FurnitureStore-backend'
metadata = parse_java_files(directory)

with open('metadata.json', 'w') as json_file:
    json.dump(metadata, json_file, indent=4)

actual_data = json.dumps(metadata, indent=4)
print(actual_data)

[
    {
        "class_name": "FurnitureStore",
        "file_path": "/content/FurnitureStore-backend/furniturestore/src/main/java/com/furniturestore/FurnitureStore.java",
        "methods": [
            {
                "name": "main",
                "parameters": [
                    "args"
                ]
            }
        ],
        "attributes": [],
        "dependencies": [
            "SpringApplication",
            "SpringBootApplication"
        ]
    },
    {
        "class_name": "OrderDetailController",
        "file_path": "/content/FurnitureStore-backend/furniturestore/src/main/java/com/furniturestore/controller/OrderDetailController.java",
        "methods": [
            {
                "name": "placeOrder",
                "parameters": [
                    "isSingleProductCheckout",
                    "orderInput"
                ]
            },
            {
                "name": "getOrderDetails",
                "parameters": []
            },
   

In [82]:
from langchain_astradb import AstraDBVectorStore
from langchain_core.documents import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.colab import userdata
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=userdata.get('GOOGLE_API_KEY1'))
vstore = AstraDBVectorStore(
    collection_name="test_hello1",
    embedding=embeddings,
    token=userdata.get("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=userdata.get("ASTRA_DB_API_ENDPOINT"),
)


In [83]:
def insert_data_to_astra_langchain(collection, data):
    try:
        data = json.loads(data)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return

    docs = []
    for entry in data:
        metadata = {
            "file_path": entry["file_path"],
            "class_name": entry["class_name"],
        }
        if entry.get("attributes"):
            for attribute in entry["attributes"]:
                metadata[attribute] =entry["class_name"]
        if entry.get("dependencies"):
            for dependency in entry["dependencies"]:
                metadata[dependency] = entry["class_name"]

        page_content = f"Class: {entry['class_name']}\nMethods: {', '.join([method['name'] for method in entry['methods']])}"
        doc = Document(page_content=page_content, metadata=metadata)
        docs.append(doc)

    try:
        inserted_ids = vstore.add_documents(docs)
        print(f"Inserted {len(inserted_ids)} documents.")
    except Exception as e:
        print(f"Error inserting documents: {e}")

insert_data_to_astra_langchain(vstore, actual_data)

Inserted 59 documents.


In [84]:
results = vstore.similarity_search("User update", k=3)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Class: UserDao
Methods:  [{'file_path': '/content/FurnitureStore-backend/furniturestore/src/main/java/com/furniturestore/dao/UserDao.java', 'class_name': 'UserDao', 'CrudRepository': 'UserDao', 'Repository': 'UserDao', 'User': 'UserDao'}]
* Class: User
Methods:  [{'file_path': '/content/FurnitureStore-backend/furniturestore/src/main/java/com/furniturestore/entity/User.java', 'class_name': 'User', 'userName': 'User', 'userFirstName': 'User', 'userLastName': 'User', 'userPassword': 'User', 'role': 'User', 'Set': 'User', 'CascadeType': 'User', 'Entity': 'User', 'FetchType': 'User', 'Id': 'User', 'JoinColumn': 'User', 'JoinTable': 'User', 'ManyToMany': 'User', 'Data': 'User'}]
* Class: UserServiceImplTests
Methods: init, testRegisterNewUser, testRegisterNewUserThrowsException [{'file_path': '/content/FurnitureStore-backend/furniturestore/src/test/java/com/furniturestore/servicetest/UserServiceImplTests.java', 'class_name': 'UserServiceImplTests', 'userService': 'UserServiceImplTests', 'u

In [85]:
reteriver=vstore.as_retriever(search_kwargs={"k": 10})
retrieved_docs = reteriver.invoke("I want to update user?")
print(retrieved_docs)

[Document(id='5d499101227b41a898dd84791f9732b6', metadata={'file_path': '/content/FurnitureStore-backend/furniturestore/src/main/java/com/furniturestore/dao/CartDao.java', 'class_name': 'CartDao', 'List': 'CartDao', 'CrudRepository': 'CartDao', 'Repository': 'CartDao', 'Cart': 'CartDao', 'User': 'CartDao'}, page_content='Class: CartDao\nMethods: findByUser'), Document(id='f1cc9abb50db49ed811412c6b473df09', metadata={'file_path': '/content/FurnitureStore-backend/furniturestore/src/main/java/com/furniturestore/dao/OrderDetailDao.java', 'class_name': 'OrderDetailDao', 'List': 'OrderDetailDao', 'CrudRepository': 'OrderDetailDao', 'Repository': 'OrderDetailDao', 'OrderDetail': 'OrderDetailDao', 'User': 'OrderDetailDao'}, page_content='Class: OrderDetailDao\nMethods: findByUser'), Document(id='4f7c8bd22a724747b2015f55b9322a3b', metadata={'file_path': '/content/FurnitureStore-backend/furniturestore/src/main/java/com/furniturestore/dao/UserDao.java', 'class_name': 'UserDao', 'CrudRepository': 

In [92]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",temperature=0.3,api_key=userdata.get('GOOGLE_API_KEY1'))

In [87]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an java developer assistant for an springboot project . "
    "Use the following pieces of retrieved context to get the understanding for related classes and answer for "
    "the question. If you don't know the answer, say that you "
    "don't know. provide the required changes needed clearly and also provide sample code changes"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [88]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(reteriver, question_answer_chain)

In [95]:
response = rag_chain.invoke({"input": "i want to create micro service arch for my project?"})
print("Response:", response["answer"])

Response: Okay, I can provide guidance on how to approach creating a microservice architecture for your Spring Boot project, based on the classes you've provided.

**General Strategy**

The key idea is to break down your application into smaller, independent services that communicate with each other.  Based on your existing classes, a reasonable starting point would be to separate concerns like this:

1.  **Product Service:**  Handles everything related to products.
    *   Includes: `ProductService`, `ProductServiceImpl`, `ProductServiceImplTests`, `Product`, and related repositories/data models.
2.  **Order Service:**  Manages order placement and details.
    *   Includes: `OrderDetailService`, `OrderDetailServiceImpl`, `OrderDetailServiceImplTests`, `OrderDetail`, and related repositories/data models.
3.  **User Service:**  Deals with user management and authentication.
    *   Includes: `UserService`, `UserServiceImpl`, `RoleServiceImplTests`, `Role`, `User`, and related repositori